# Connecting with the Credo AI Governance App

Connecting with the Credo AI Governance App is straightforward using Lens. This notebook is a comprehensive overview of how you you can interact with the Governance App.

## Setup

**Find the code**

This notebook can be found on [github](https://github.com/credo-ai/credoai_lens/blob/develop/docs/notebooks/governance_integration.ipynb).

**Config File**

First, ensure you have a config file somewhere on your computer named ".credoconfig". The default location where Lens will look for it in your home directory (`~/.credoconfig`). The structure of the config should look like this

```
TENANT=<tenant name> # Example: credoai
CREDO_URL=<your credo url> 
API_KEY=<your api key> # Example: JSMmd26...
```

This config gives Lens API access to the Governance App. A config file can be downloaded from the web app. MySettings -> Token

**Governance App Setup**
Connecting Lens to the Governance App requires that you have already defined a Use-Case and registered a model to that use-case. That model serves as the endpoint for Lens.

**Data + Model Preparation (before Lens)**

Some quick setup. This script reflects all of your datascience work before assessment and integration with Credo AI.

Here we have a gradient boosted classifier trained on the UCI Credit Card Default Dataset.

In [1]:
# model and df are defined by this script
%run training_script.py

### Imports

In [2]:
# Base Lens imports
import credoai.lens as cl
# set default format for image displays. Change to 'png' if 'svg' is failing
%config InlineBackend.figure_formats = ['svg']

## Integration Quickstart

Building off of the [quickstart notebook](https://credoai-lens.readthedocs.io/en/latest/notebooks/quickstart.html), let's look at how you can export to a Use Case on the Governance App. We only have to add two new lines.

In [3]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          data=df,
                          sensitive_feature_keys=['SEX'],
                          label_key='target'
                          )

In [ ]:
# The below will fail unless these IDs are defined!
use_case_id = 'x'  # your use case id can be found in the Governance App
model_id = 'x'  # your model id can be found in the Governance App

# when interacting with the Governace Platform, the spec_destination looks like this.
spec_destination = f'use_cases/{use_case_id}/models/{model_id}/assessment_spec' 

lens = cl.Lens(model=credo_model,
               data=credo_data,
               governance=spec_destination # <- NEW LINE!
               )

lens.run_assessments().export() # <- Added export

Those two lines did a number of things under the hood.

* First a [CredoGovernance Artifact](#Credo-Governance-Artifact) was created with the credo-url provided.
* Next the datasets were registered:
    * The dataset was registered to the Governanc App
    * The dataset was registered as validation data for the associated model and use-case.
* Next an assessment plan was downloaded and updated:
    * An assessment plan was downloaded for the model (if it exists).
    * The assessment plan was updated with the assessment_plan specified in code here.
* Finally, when `export` was called the assessment was exported to the model under the use-case.

## In-Depth Overview

### Credo Governance Artifact

Lens connects to the Governance App via a `CredoGovernance` object. All you need to do is provide a `spec-destination`. The `spec-destination` can be one of two things:
* A url for the spec in the AI Governance app (which can be found in the Governance App, or in an email requesting a model assessment))
* A file path to the spec after it was downloaded from the AI Governance App (an example assessment_plan.json is included in the repo for this package and will be referenced below).

**The CredoGovernance object has a few functionalities.**

* Retrieving an alignment spec from the Credo AI Governance App. The alignment spec is either downloaded straight from the Governance App based on a credo_url, or supplied as a json file for air gap deployment. 
    * This alignment spec includes an assessment plan, which can be supplemented (or overwritten) with an assessment plan you define in code and pass to Lens.
* Registering artifacts (e.g., datasets) to the Governance App.
* Encoding the IDs of governance objects so Lens can easily export metrics and reports.

### Retrieving an alignment spec

Here we use a local assessment spec in the Credo AI Lens repository. In general, you'd only do this in an air gapped environment. More often, you'd provide an API url that connects Lens with the Governance Platform. That API Url has the following form:

```use_cases/{use_case_id}/models/{model_id}/assessment_spec```


Regardless, the assessment spec is a primary way in which the Governance App communicates with Lens and defines an assessment strategy.

In [4]:
# Here we reference a local assessment plan, rather than providing a url!
# Normally you would only do this in an air-gap environment
credo_gov = cl.CredoGovernance(spec_destination = 'assessment_spec.json')

# one aspect of the assessment spec is an assessment plan...
credo_gov.get_assessment_plan()

defaultdict(dict, {'Performance': {'metrics': ['precision_score']}})

### Registering artifacts using CredoGovernance

Models and datasets can be registered programatically, with no assessment spec needed. Only a use-case ID must be provided.

Assessment templates (assessment plan blueprints saved in the Governance Platform) can be programatically applied to a model once it has been registered.

In [ ]:
credo_gov = cl.CredoGovernance()
# the use case must be defined before registering anything!
credo_gov.use_case_id = "Use-Case-Id"

# registering a model and dataset
credo_gov.register(model_name = "My Model",
                   dataset_name = "My Dataset")

# applying an assessment template to the registered model
# Note! This assessment template must exist in the Governance Platform
credo_gov.register(assessment_template = "My Template")

# Registering a model, followed by applying an assessment template to it
credo_gov.register(model_name = "My Model",
                   assessment_template = "My Template")

### Interacting with Credo AI's Governance App from Lens

In addition to the model and data, we can pass the CredoGovernance object. This is how Lens knows which Use Case, model and/or data to interacts with. If you are running assessments on multiple models, you'll create a separate `CredoGovernance` object (and separate Lens instances) for each.

You can also pass the assessment spec location and nothing else. In this case a CredoGovernance object will be created using that destination.

**Note**

This use assumes that a model is _already_ registered to the governance app with an assessment plan. You can easily do this in the Governance App, or by registering the model first programatically, as shown.

In [5]:
from credoai.assessment import FairnessAssessment, PerformanceAssessment
lens = cl.Lens(model=credo_model,
               data=credo_data,
               governance=credo_gov,
               assessments = [PerformanceAssessment, FairnessAssessment])

# After running, you just have to export and the metrics 
# will be sent to the Governance App
results = lens.run_assessments().get_results()

INFO:absl:Running assessment: Performance
INFO:absl:Running assessment: Fairness


## Air Gap Environment

If you cannot interact with Credo AI's Governance App via the internet, this section is for you!

Instead of Lens automating the communication between your assessment environment and governance, you will instead have to download an assessment_spec, provided it to Lens, and then upload the results assessments. The demo has actually already been using a local example spec included in the github repo.

### Getting the assessment spec
The asset that you have to get _from_ the governance app is the assessment spec. This can be found under your use case's "Risk Assessment" tab inside "Align". Once you download the assessments spec, you can read it with a CredoGovernance object by changing one argument.

**Important Note On Data Registration in an air-gapped environment** 

When interacting with Credo AI's API you _only_ have to register a Use Case and Model within the app - nothing neds to be done relating to data. Lens will handle registering your training/assessment data to the app.

However, when in an air-gapped environment you WILL need to register the training and validation data in the app first. Otherwise Lens will fail.


In [ ]:
gov = cl.CredoGovernance(spec_destination='{path to assessment spec}')

Following that, you can pass the governance object to Lens as normal.

### Exporting Results

Lens can export assessment results to file as a json object. This is generally useful if you want to store your results locally, but particularly useful if you are in an air gapped environment where you cannot access Credo AI's Governance App directly from your development environment.

Doing this only requires a one line change in our current flow. We change the `export` argument from `credoai` to a local path:


In [ ]:
lens.run_assessments().export('{path where assessments should be exported}')

Running the above will create a folder that you specified, if it doesn't exist. It will then create a json file with the assessment results, which can be uploaded to Credo AI's Governance App. This json file can be uploaded by going to the assessments of your **Use-Case** and pressing the purple "upload assessment" button.